<a href="https://colab.research.google.com/github/costaljorge/AlgoritmoCN2/blob/main/Aprendizado_SemiSupervisionado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### O Orange é uma plataforma de código aberto para mineração de dados e análise de dados visuais. Oferece uma interface intuitiva e flexível, permitindo aos usuários realizar tarefas de mineração de dados complexas de forma eficiente. Com uma ampla variedade de ferramentas e algoritmos pré-construídos, o Orange permite explorar dados, realizar pré-processamento, visualizar resultados e construir modelos de machine learning de forma rápida e fácil. Sua abordagem visual facilita a compreensão dos dados e dos resultados dos modelos, tornando-o uma escolha popular tanto para iniciantes quanto para especialistas em ciência de dados. O Algoritmo CN2 não existe no sklearn, mas voce consegue roda-lo através do Orange.

In [ ]:
#Instalando o Orange

!pip install Orange3

In [ ]:
# Importando as bibliotecas necessárias

from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
import pandas as pd


# Lendo o DataFrame

df = pd.read_excel('/content/Filmes.xlsx')

# Calculando a matriz de distância com a distância de Hamming

distancia = pdist(df.values, metric='hamming')
matriz_distancia = squareform(distancia)

# Aplicando o algoritmo de Clustering Hierárquico

cluster = AgglomerativeClustering(n_clusters=3, affinity='precomputed', linkage='average')
predicoes = cluster.fit_predict(matriz_distancia)

# Preparando os clusters do Dataframe

df['cluster'] = predicoes
aux = {0: 'Classe_assite_1', 1: 'Classe_assite_2', 2: 'Classe_assite_3'}
df['cluster'] = df['cluster'].replace(aux)
df.set_index(df.columns[0], inplace=True)
df.rename(columns={'cluster': 'c#cluster'}, inplace=True)
df.to_excel('Cluster_filmes.xlsx')




In [ ]:
import Orange

# Carregar o conjunto de dados
data = Orange.data.Table('/content/Cluster_filmes.xlsx')
cn2=Orange.classification.rules.CN2Learner()
regras=cn2(data)

In [ ]:
# Imprimindo as regras

for rule in regras.rule_list:
    print(rule)

In [ ]:
#Imprindo as regras de acordo com uma certa probabilidade

for i in regras.rule_list:
  if i.probabilities[1] >= .65:
    print(i,"------------------",i.probabilities[1])

In [98]:
#Imprindo as regras em um arquivo texto

with open("saida.txt", "w") as arquivo:
  for i in regras.rule_list:
    if i.probabilities[1] >= .65:
      # Escreva a variável no arquivo
      arquivo.write(str(i) + "\n")

In [ ]:
# Calculando a acurácia do modelo

acuracia = Orange.evaluation.testing.TestOnTestData(data, data, [lambda testdata: regras])
print('Acurácia:', Orange.evaluation.CA(acuracia)[0]*100)